### library imports and feature selection

In [22]:
import pandas as pd
import numpy as np
import torch
import os 

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
#from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch

import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

import gensim

import torch
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.feature_extraction.text import TfidfVectorizer

import spacy
import string

random_state = 42

'''
joints = [
    "(0, 0)", "(0, 1)", "(0, 2)", "(1, 0)", "(1, 1)", "(1, 2)", "(2, 0)", "(2, 1)", "(2, 2)", 
    "(3, 0)", "(3, 1)", "(3, 2)", "(4, 0)", "(4, 1)", "(4, 2)", "(5, 0)", "(5, 1)", "(5, 2)", 
    "(6, 0)", "(6, 1)", "(6, 2)", "(7, 0)", "(7, 1)", "(7, 2)", "(8, 0)", "(8, 1)", "(8, 2)", 
    "(9, 0)", "(9, 1)", "(9, 2)", "(10, 0)", "(10, 1)", "(10, 2)", "(11, 0)", "(11, 1)", "(11, 2)", 
    "(12, 0)", "(12, 1)", "(12, 2)", "(13, 0)", "(13, 1)", "(13, 2)", "(14, 0)", "(14, 1)", "(14, 2)"
]
'''

joints = [
    "(0, 0)", "(0, 1)", "(1, 0)", "(1, 1)", "(2, 0)", "(2, 1)", 
    "(3, 0)", "(3, 1)", "(4, 0)", "(4, 1)", "(5, 0)", "(5, 1)", 
    "(6, 0)", "(6, 1)", "(7, 0)", "(7, 1)", "(8, 0)", "(8, 1)", 
    "(9, 0)", "(9, 1)", "(10, 0)", "(10, 1)", "(11, 0)", "(11, 1)", 
    "(12, 0)", "(12, 1)", "(13, 0)", "(13, 1)", "(14, 0)", "(14, 1)"
]

audio_col = [
    'F0semitoneFrom27.5Hz_sma3nz_amean','F1amplitudeLogRelF0_sma3nz_amean','F1bandwidth_sma3nz_amean','F1frequency_sma3nz_amean',
    'F2amplitudeLogRelF0_sma3nz_amean','F2bandwidth_sma3nz_amean','F2frequency_sma3nz_amean','F3amplitudeLogRelF0_sma3nz_amean',
    'F3bandwidth_sma3nz_amean','F3frequency_sma3nz_amean','HNRdBACF_sma3nz_amean','alphaRatioV_sma3nz_amean',
    'hammarbergIndexV_sma3nz_amean','jitterLocal_sma3nz_amean','logRelF0-H1-A3_sma3nz_amean','logRelF0-H1-H2_sma3nz_amean',
    'loudness_sma3_amean','mfcc1_sma3_amean','mfcc2_sma3_amean','mfcc3_sma3_amean','mfcc4_sma3_amean','shimmerLocaldB_sma3nz_amean',
    'slopeV0-500_sma3nz_amean','slopeV500-1500_sma3nz_amean','spectralFlux_sma3_amean'
]

# 12 DLD, 7 TD
#DLD_child = ['DLD_bike_ban108.json','DLD_bike_chi108.json', 'DLD_bike_dou108.json', 'DLD_bike_fag108.json', 'DLD_bike_ken108.json', 'DLD_bike_kil108.json', 'DLD_bike_mcb108.json', 'DLD_bike_mel108.json', 'DLD_bike_peq108.json', 'DLD_bike_phi108.json', 'DLD_bike_ros108.json', 'DLD_bike_sul108.json']
#TD_child = ['TD_bike_aik108.json', 'TD_bike_ham108.json', 'TD_bike_lav108.json', 'TD_bike_pem108.json', 'TD_bike_pia108.json', 'TD_bike_tro108.json', 'TD_bike_spo108.json']

# no: gra, ben
# yes: rob, son

# 16 DLD, 11 TD
# DLD_child = ['DLD_bike_mcg108.json','DLD_bike_bic108.json', 'DLD_bike_rob108.json', 'DLD_bike_son108.json', 'DLD_bike_ban108.json','DLD_bike_chi108.json', 'DLD_bike_dou108.json', 'DLD_bike_fag108.json', 'DLD_bike_ken108.json', 'DLD_bike_kil108.json', 'DLD_bike_mcb108.json', 'DLD_bike_mel108.json', 'DLD_bike_peq108.json', 'DLD_bike_phi108.json', 'DLD_bike_ros108.json', 'DLD_bike_sul108.json']
# TD_child = ['TD_bike_ker108.json','TD_bike_cla108.json','TD_bike_ser108.json', 'TD_bike_mah108.json','TD_bike_aik108.json', 'TD_bike_ham108.json', 'TD_bike_lav108.json', 'TD_bike_pem108.json', 'TD_bike_pia108.json', 'TD_bike_tro108.json', 'TD_bike_spo108.json']

DLD_child = ['mcg108.json','bic108.json', 'rob108.json', 'son108.json', 'ban108.json','chi108.json', 'dou108.json', 'fag108.json', 'ken108.json', 'kil108.json', 'mcb108.json', 'mel108.json', 'peq108.json', 'phi108.json', 'ros108.json', 'sul108.json']
TD_child = ['ker108.json','cla108.json','ser108.json', 'mah108.json','aik108.json', 'ham108.json', 'lav108.json', 'pem108.json', 'pia108.json', 'tro108.json', 'spo108.json']



### initalize fasttext api for embeddings

In [23]:
# select pretrained word vectorization 
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))
wv = api.load('fasttext-wiki-news-subwords-300')

nlp = spacy.load("en_core_web_sm")
punctuations = string.punctuation

# tokenizer to extract keywords  
def spacy_tokenizer(sentence):

    # get english words from sentence
    doc = nlp(sentence)

    # lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # removing stop words
    mytokens = [ word for word in mytokens if  word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

# aggreate all words into single representation
def sent_vec(sent):

    # get vector size and create dummy holder
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)

    # iterate through all words to add then average 
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr

    return wv_res

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


# Tokenization
 this version of data preprocess will be tokenized and its temporal dimension will be removed by averaging all values within the same time stamp

 this will tokenization the two class: typically developing and late talkers (TD VS DLD)# code for all 3 features 

In [28]:
# IF YOU GO BACK TO SINGLE EXPERIEMENT TEST, REMOVE THE [:-11]
# if you want to combine dataset, remove the "bike" in filename part


# set token size here:
chunk_tokens = 50

# *** THE FOLLOWING TOKENIZES THE TD & DLD DATA TO CREATE DATASET ***

# chunk names = chunked person, duration  = duration of chunk
#TD_Tokens_path = 'C:/USF/RESEARCH LABS/CSAIL/aphsia_baseline/joints_preprocessing/TD_normalized'
TD_Tokens_path = 'C:/WORK/RESEARCH LABS/CSAIL/aphsia_baseline/dataset/TD_tokens_plus'
chunk_names_TD = []
duration_TD = []

# create chunks for each feature type
TD_chunk_audio = []
TD_chunk_text = []
TD_chunk_joints = []
transcript_TD = []


for filename in os.listdir(TD_Tokens_path):

    sub_transcript_TD = [] 
    # if not in TD skip
    if filename[-11:] not in TD_child:
        continue 

    if "retell" not in filename:
        continue

    # read file
    child = filename[:-5]
    file_path = os.path.join(TD_Tokens_path, filename)
    df = pd.read_json(file_path)

    # remove empty chunks and fill nans with 0s
    df = df[df['chunk_id'].notna()]
    df = df.fillna(0, inplace=False)

    # create vector encodings of words
    df['mod_token'] = df['token'].apply(spacy_tokenizer)
    df['vec'] = df['mod_token'].apply(sent_vec)


    # get number of rows 
    number_of_rows = df.shape[0]


    # create start and end time stamps for duration calcuation
    start = df['start'].to_numpy()
    end = df['end'].to_numpy()

    for index, data in df.iterrows():
        sub_transcript_TD.append(data['token'])



    # create separate dfs for each features (audio, joints, text)
    df_audio = df[audio_col]
    df_joints = df[joints]
    df_text = df['vec']
    
    audio_data = df_audio.to_numpy()
    joint_data = df_joints.to_numpy()
    text_data = df_text.to_list()


    
    # create lower and upper bounds for tokenization chunking
    lower_bound = 0
    upper_bound = 1
    for i in range(number_of_rows//chunk_tokens):
        # create lower and upper bounds for tokenization
        lower = lower_bound * chunk_tokens
        upper = (upper_bound * chunk_tokens)

        # get range, list is exclusive of upper bound for all features 
        TD_chunk_audio.append(audio_data[lower:upper].tolist())
        TD_chunk_joints.append(joint_data[lower:upper].tolist())
        TD_chunk_text.append(text_data[lower:upper])

        # get sub transcript per each token
        transcript_TD.append(sub_transcript_TD[lower:upper])

        chunk_names_TD.append(child)
        duration_TD.append(end[upper -1 ] - start[lower])

        # increment lower and upper bounds
        lower_bound += 1
        upper_bound += 1

# chunk names = chunked person, duration  = duration of chunk
#DLD_Tokens_path = 'C:/USF/RESEARCH LABS/CSAIL/aphsia_baseline/joints_preprocessing/DLD_normalized'
DLD_Tokens_path = 'C:/WORK/RESEARCH LABS/CSAIL/aphsia_baseline/dataset/DLD_tokens_plus'
chunk_names_DLD = []
duration_DLD = []

# create chunks for each feature type
DLD_chunk_audio = []
DLD_chunk_text = []
DLD_chunk_joints = []
transcript_DLD = []


for filename in os.listdir(DLD_Tokens_path):

    sub_transcript_TD = [] 
    # if not in TD skip
    if filename[-11:] not in DLD_child:
        continue 

    if "retell" not in filename:
        continue

    # read file
    child = filename[:-5]
    file_path = os.path.join(DLD_Tokens_path, filename)
    df = pd.read_json(file_path)

    # remove empty chunks and fill nans with 0s
    df = df[df['chunk_id'].notna()]
    df = df.fillna(0, inplace=False)

    # create vector encodings of words
    df['mod_token'] = df['token'].apply(spacy_tokenizer)
    df['vec'] = df['mod_token'].apply(sent_vec)


    # get number of rows 
    number_of_rows = df.shape[0]


    # create start and end time stamps for duration calcuation
    start = df['start'].to_numpy()
    end = df['end'].to_numpy()

    for index, data in df.iterrows():
        sub_transcript_TD.append(data['token'])



    # create separate dfs for each features (audio, joints, text)
    df_audio = df[audio_col]
    df_joints = df[joints]
    df_text = df['vec']
    
    audio_data = df_audio.to_numpy()
    joint_data = df_joints.to_numpy()
    text_data = df_text.to_list()


    
    # create lower and upper bounds for tokenization chunking
    lower_bound = 0
    upper_bound = 1
    for i in range(number_of_rows//chunk_tokens):
        # create lower and upper bounds for tokenization
        lower = lower_bound * chunk_tokens
        upper = (upper_bound * chunk_tokens)

        # get range, list is exclusive of upper bound for all features 
        DLD_chunk_audio.append(audio_data[lower:upper].tolist())
        DLD_chunk_joints.append(joint_data[lower:upper].tolist())
        DLD_chunk_text.append(text_data[lower:upper])

        # get sub transcript per each token
        transcript_DLD.append(sub_transcript_TD[lower:upper])

        chunk_names_DLD.append(child)
        duration_DLD.append(end[upper -1 ] - start[lower])

        # increment lower and upper bounds
        lower_bound += 1
        upper_bound += 1



TD_audio = torch.tensor(TD_chunk_audio)
TD_audio = TD_audio.mean(dim=1)
DLD_audio = torch.tensor(DLD_chunk_audio)
DLD_audio = DLD_audio.mean(dim=1)
audio = torch.cat((TD_audio, DLD_audio), dim=0)

TD_text = torch.tensor(TD_chunk_text)
TD_text = TD_text.mean(dim=1)
DLD_text = torch.tensor(DLD_chunk_text)
DLD_text = DLD_text.mean(dim=1)
text = torch.cat((TD_text, DLD_text), dim=0)

TD_joints = torch.tensor(TD_chunk_joints)
TD_joints = TD_joints.mean(dim=1)
DLD_joints = torch.tensor(DLD_chunk_joints)
DLD_joints = DLD_joints.mean(dim=1)
joint = torch.cat((TD_joints, DLD_joints), dim=0)

y_TD = torch.zeros(len(chunk_names_TD))
y_DLD = torch.ones(len(chunk_names_DLD))
y = torch.cat((y_TD, y_DLD), dim=0)

y_names = chunk_names_TD + chunk_names_DLD
transcript = transcript_TD + transcript_DLD
y = [int(y) for y in y]


### Save each modalities of data to npy and json files

apply scaler to joint and visual (text is already scaled)

In [29]:
scaler = StandardScaler()


df_data = pd.DataFrame({
    "joint_feature_scaled": scaler.fit_transform(joint).tolist(),
    "audio_feature_scaled": scaler.fit_transform(audio).tolist(),
    "text_feature_scaled": text.tolist(),  # Original features
    "transcript": transcript,
    "label": y,
    "name": y_names
})

for index, data in df_data.iterrows():
    full = " ".join(data['transcript'])
    df_data.at[index, 'full_transcript'] = full

df_data['label'], df_data['full_transcript'] = df_data['full_transcript'], df_data['label']
df_data.rename(columns={'label': 'full_transcript', 'full_transcript': 'label'}, inplace=True)
print()

In [31]:
df_data.to_json("data_50_retelll.json")